# Units Added by Year

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

from sqlalchemy import create_engine
from pysandag.database import get_connection_string

%matplotlib inline

db_connection_string = get_connection_string('..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)

get max run id

In [ ]:
run_id_sql = '''
SELECT max(run_id)
  FROM [urbansim].[urbansim].[urbansim_lite_output]
'''
run_id_df = pd.read_sql(run_id_sql, mssql_engine)
run_id = int(run_id_df.values)
print(run_id)

get housing units needed from regionwide forecast

In [ ]:
housing_unit_sql = '''select  yr,
    housing_units1, households,housing_units_add 
    from [isam].[economic_output].[urbansim_housing_units]'''
hu_df =  pd.read_sql(housing_unit_sql, mssql_engine,index_col ='yr')
hu_df['total_housing_units_add'] = hu_df.housing_units_add.cumsum()
hu_df.rename(columns = {'housing_units_add': 'Housing units to add'}, inplace=True)
hu_df_to_plot = hu_df[['Housing units to add']].copy()

get housing units added from latest run

In [ ]:
hs_change__by_yr_sql = '''
    SELECT sum(units_added) as housing_units_added, year_simulation
      FROM urbansim.urbansim.urbansim_lite_output o 
      JOIN urbansim.urbansim.parcel p on p.parcel_id = o.parcel_id
      JOIN urbansim.ref.jurisdiction j on p.jurisdiction_id = j.jurisdiction_id
     WHERE run_id =  %s
  GROUP BY year_simulation
  ORDER BY year_simulation'''
hs_change__by_yr_sql = hs_change__by_yr_sql % run_id
units_added_by_yr = pd.read_sql(hs_change__by_yr_sql,mssql_engine,index_col='year_simulation')
units_added_by_yr.rename(columns = {'housing_units_added': 'Housing units added'}, inplace=True)

get housing units by source

In [ ]:
units_added_sched_dev_sql = '''
 SELECT sum(units_added) as housing_units_added,year_simulation,o.source
      FROM [urbansim].[urbansim].[urbansim_lite_output] o
      JOIN [urbansim].[urbansim].[parcel] p on p.parcel_id = o.parcel_id
      JOIN urbansim.ref.jurisdiction j on p.jurisdiction_id = j.jurisdiction_id
     WHERE run_id = %s
  GROUP BY o.source,year_simulation
  ORDER BY year_simulation,o.source
'''
units_added_sched_dev_sql = units_added_sched_dev_sql % run_id
units_added_sched_dev_df =  pd.read_sql(units_added_sched_dev_sql,mssql_engine)
units_added_by_source = units_added_sched_dev_df.pivot(index='year_simulation', columns='source', values='housing_units_added')

plot housing units needed and added by simulation

In [ ]:
fig, axes = plt.subplots(3, 1)
hu_df_to_plot.plot(style='.-',ax=axes.flat[0],figsize=(10,15),color='b')
units_added_by_yr.plot(style='.-',ax=axes.flat[1],figsize=(10,15),color=['darkorange','b'])
units_added_by_source.plot(style='.-',stacked=True,kind='bar',ax=axes.flat[2],figsize=(10,15),color=['red','purple','green'])
# units_added_by_source.plot(stacked=True,kind='bar');
# format
fmt = '{x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
axes.flat[0].set_xlabel("Simulation Year",size =16)
axes.flat[0].set_ylabel("Housing Units",size =16)
axes.flat[0].set_title('Housing Units Needed by Year\n demographic_simulation_id=2001',size=16) 
axes.flat[0].yaxis.set_major_formatter(tick) 
axes.flat[0].legend(fontsize=14)
axes.flat[1].set_xlabel("Simulation Year",size =16)
axes.flat[1].set_ylabel("Housing Units",size =16)
axes.flat[1].set_title('Housing Units Added by Year',size=16) 
axes.flat[1].yaxis.set_major_formatter(tick) 
axes.flat[2].legend(fontsize=14)
axes.flat[2].set_xlabel("Simulation Year",size =16)
axes.flat[2].set_ylabel("Housing Units",size =16)
axes.flat[2].set_title('Housing Units Added by Source',size=16) 
axes.flat[2].yaxis.set_major_formatter(tick) 
axes.flat[2].legend(fontsize=14)
# format
plt.xticks(rotation=25)
plt.tight_layout(pad=0.5, w_pad=0.6, h_pad=2)
plt.show()